In [1]:
import pandas as pd
import numpy as np


In [2]:
# configuration setup

BASE_DIR = "~/Research/wheelchair/data/raw/Max"
OUTPUT_DIR = "~/Research/wheelchair/data/processed/"
initials = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS"]
materials = ["PLA", "HYB"]

In [3]:
file_paths = [
    {
        "input": f"{BASE_DIR}/{material}/{initial}25{material}.csv",
        "output": f"{OUTPUT_DIR}/{material}/{initial}25{material}_kinetics.csv"
    }

    for initial in initials
    for material in materials
]

In [4]:
# more helper functions
# note: these need to be run on the original dataframe, unfiltered

"""
Calculates the contact angle (deg)
  (Max theta_cop_[deg]) – (Min theta_cop_[deg])

  x: series data containing the angle
"""
def contact_angle(x):
    if len(x) == 0:
        return np.nan
    else:
        return x.dropna().max() - x.dropna().min()
    

"""
Calculates the contact ratio %
Contact time relative to the duration of one cycle
Params:
  x - the series data for the angle
"""
def calculate_contact_time(x):
    return 100*x.notna().mean()

#=============================== Calculation functions for the theta_cop =========================
def peak_torque_angle(side): # when the torque peaks
    idx = df[df['moment_z[W]'] == max(df['moment_z[W]'])]
    angle_col = f"theta_cop_{side}[deg]"
    return df.loc[idx, angle_col]

def peak_power_angle(side): # when the power peaks
    idx = df[df['power_z[W]'] == max(df['power_z[W]'])]
    angle_col = f"theta_cop_{side}[deg]"
    return df.loc[idx, angle_col]

def peak_tangential_force_angle(side): # when the tangential force is highest
    tangential_force_col = f"tangential_force_{side}[N]"
    idx = df[df[tangential_force_col] == max(df[tangential_force_col])]
    angle_col = f"theta_cop_{side}[deg]"
    return df.loc[idx, angle_col]

def angle_of_radial_force_change(df, side):
    # find when the radial force flips from positive to negative
    angle_col = f"theta_cop_{side}[deg]"
    radial_force_col = f"radial_force_{side}[N]"
    idx = first_pos_to_neg(df, radial_force_col)
    return df.loc[idx, angle_col]

def angle_of_axle_force_change(x):
    angle_col = f"theta_cop_{side}[deg]"
    axel_force_col = f"axle_force_{side}[N]"
    idx = first_pos_to_neg(df, axel_force_col)
    return df.loc[idx, angle_col]

def first_pos_to_neg(df, col):
    # Convert series to numpy array for speed
    vals = df[col].values
    time = df['time[sec]'].values
    
    # Loop through values to detect first positive -> negative
    for i in range(1, len(vals)):
        if vals[i-1] > 0 and vals[i] < 0:
            return time[i]
    
    # If no flip found
    return None

#============= A summary statistic per each person that needs to be calculated =======================

def calculate_stroke_frequency(df):
    TOTAL_CYCLES = 25
    
    total_time = df['time[sec]'].max() -  df['time[sec]'].min()
    return TOTAL_CYCLES / total_time
    

In [5]:
# get the speed, contact angle and contact time
def get_contact_params():
    sides = ["L", "R"]
    return {
    "max_speed[km/h]": ("speed_R[km/h]", "max"),
    "contact_angle_R[deg]": ("theta_cop_R[deg]", contact_angle),
    "contact_angle_L[deg]": ("theta_cop_L[deg]", contact_angle),
    "contact_time_R[deg]": ("theta_cop_R[deg]", calculate_contact_time),
    "contact_time_L[deg]": ("theta_cop_L[deg]", calculate_contact_time),
}

In [6]:
# Helper function to compute key metrics, for when the glove is still touching
def compute_cycle_metrics(df, side):
    """
    Compute key angles for each cycle where various metrics peak or change sign.
    """
    angle_col = f"theta_cop_{side}[deg]"
    torque_col = f"moment_z_{side}[Nm]"
    tangential_force_col = f"tangential_force_{side}[N]"
    radial_force_col = f"radial_force_{side}[N]"
    axle_force_col = f"axle_force_{side}[N]"

    df_copy = df[df[angle_col].notna()] 
    def peak_torque_angle(group):
        idx = group[torque_col].idxmax()
        return group.loc[idx, angle_col]
    
    def peak_power_angle(group):
        idx = group['power_z[W]'].idxmax()
        return group.loc[idx, angle_col]
    
    def peak_tangential_force_angle(group):
        idx = group[tangential_force_col].idxmax()
        return group.loc[idx, angle_col]
    
    def angle_of_radial_force_change(group):
        vals = group[radial_force_col].values
        angles = group[angle_col].values
        for i in range(1, len(vals)):
            if vals[i-1] > 0 and vals[i] < 0:
                return angles[i]

        return None
    
    def angle_of_axle_force_change(group):
        vals = group[axle_force_col].values
        angles = group[angle_col].values
        for i in range(1, len(vals)):
            if vals[i-1] > 0 and vals[i] < 0:
                return angles[i]
        return None
    
    results = df_copy.groupby('cycle[count]').apply(lambda g: pd.Series({
        f'peak_torque_angle_{side}[Nm]': peak_torque_angle(g),
        f'peak_power_angle_{side}[W]': peak_power_angle(g),
        f'peak_tangential_force_angle_{side}[N]': peak_tangential_force_angle(g),
        f'radial_force_change_angle_{side}[N]': angle_of_radial_force_change(g),
        f'axle_force_change_angle_{side}[N]': angle_of_axle_force_change(g)
    }),
    include_groups=False)
    
    return results


In [8]:
file_paths[0]["input"]

'~/Research/wheelchair/data/raw/Max/PLA/ALP25PLA.csv'

In [9]:
df = pd.read_csv(file_paths[0]["input"])
contact_params = get_contact_params()
kinematics_summary = df.groupby("cycle[count]").agg(**contact_params)

# add power
df['power_z[W]'] = df['gyro_z_R[rad/s]']*df['moment_z_R[Nm]']

# compute the cycle metrics
cycle_metrics_R = compute_cycle_metrics(df, "R")
cycle_metrics_L = compute_cycle_metrics(df, "L")

# add all the metrics together
pd.concat([kinematics_summary, cycle_metrics_R, cycle_metrics_L], axis=1)

,max_speed[km/h],contact_angle_R[deg],contact_angle_L[deg],contact_time_R[deg],contact_time_L[deg],peak_torque_angle_R[Nm],peak_power_angle_R[W],peak_tangential_force_angle_R[N],radial_force_change_angle_R[N],axle_force_change_angle_R[N],peak_torque_angle_L[Nm],peak_power_angle_L[W],peak_tangential_force_angle_L[N],radial_force_change_angle_L[N],axle_force_change_angle_L[N]
cycle[count],,,,,,,,,,,,,,,
1.0,9.867357,128.868829,126.312692,68.333333,64.722222,89.306455,182.909307,101.314758,57.243319,NaN,87.853316,63.094732,96.803896,NaN,NaN
2.0,14.621420,182.662988,187.392446,49.808429,51.724138,89.901096,195.401238,107.564260,190.124486,NaN,89.970383,21.637179,94.204027,180.196137,NaN
3.0,18.310532,168.444841,325.874266,38.655462,45.798319,79.260007,18.060560,98.531232,NaN,NaN,107.667290,33.908305,109.170916,207.193829,NaN
4.0,21.046794,176.040560,200.275463,34.821429,41.517857,130.655705,23.162511,138.385804,NaN,NaN,108.167832,26.215297,115.111356,204.042847,NaN
5.0,23.390705,192.619522,351.738315,33.333333,37.333333,121.758840,11.378775,129.732243,62.822353,NaN,100.449230,23.248183,125.952905,208.874294,NaN
6.0,25.214879,200.324601,239.177742,27.777778,33.333333,136.795028,24.182415,138.679283,9.632199,NaN,78.613617,37.823630,128.503601,213.978148,NaN
7.0,27.008481,177.448709,198.332556,28.193833,28.634361,148.736809,15.098501,144.299824,NaN,NaN,71.743812,38.853837,123.355795,NaN,NaN
8.0,28.402089,180.764465,179.355869,27.947598,27.510917,62.996271,14.517081,140.742295,116.202342,NaN,71.251898,36.043779,134.961634,NaN,NaN
9.0,29.614808,182.888508,171.736431,24.793388,24.380165,154.909763,19.638195,154.909763,NaN,NaN,65.722732,36.566518,117.683494,NaN,NaN
